# Reformat and reorganize the radiomics data

Assumed data organization:
- $PATH/
    - raw/ (contains input data files)
    - tables/ (contains output data files)
    - figures/ (contains output image files)

In [1]:
# External packages
import pandas as pd
import pickle
import yaml
import os
import sys

In [2]:
# Local packages
sys.path.append('../src')
from ninetynine import ninetynine

In [3]:
# Read in data
with open('../config/paths.yaml','r') as file:
    paths_list = yaml.safe_load(file)
    PATH = os.path.abspath(paths_list['PATH'])

data = pd.read_csv(os.path.join(PATH,'raw','DL-Lung-Features-SARS-CoV-2-NHP-064-1-2-3_complete.csv'))

data.head()

,Study,Subject,Study Day,Study Date,File Name,Class,VoxelVolume,Maximum3DDiameter,Compactness2,MeshVolume,...,HighGrayLevelZoneEmphasis,SmallAreaEmphasis,LowGrayLevelZoneEmphasis,ZoneEntropy,SmallAreaLowGrayLevelEmphasis,Coarseness,Complexity,Strength,Contrast.1,Busyness
0,64-01,B03757,-6,2020/03/16,B03757_20200316,Infected,214687.6968,117.993344,0.063035,214813.9478,...,263.840627,0.703526,0.026156,6.520020,0.016590,0.000002,2050.728222,0.006714,0.045985,1200.810742
1,64-01,B03757,2,2020/03/24,B03757_20200324,Infected,166787.9929,106.848905,0.062049,166806.4319,...,398.640737,0.666246,0.005352,6.784057,0.003114,0.000006,2100.919283,0.024958,0.032996,235.133257
2,64-01,B03757,4,2020/03/26,B03757_20200326,Infected,179172.8874,110.873560,0.070562,179211.0040,...,390.598883,0.665000,0.006050,6.857994,0.003507,0.000005,1626.651312,0.017752,0.049790,343.480410
3,64-01,B03757,6,2020/03/28,B03757_20200328,Infected,183788.8962,114.183678,0.068394,183864.0813,...,391.586046,0.662147,0.006006,6.857685,0.003464,0.000005,1695.859842,0.018573,0.046353,343.089081
4,64-01,B03757,8,2020/03/30,B03757_20200330,Infected,192435.5003,112.927602,0.069981,192447.8780,...,365.438565,0.667409,0.006288,6.730401,0.003621,0.000005,1063.986963,0.009600,0.041377,429.421847


In [4]:
# Get rid of space in column names, annoying to deal with later
data = data.rename(columns={'Study Day':'StudyDay'})
data = data.rename(columns={'Study Date':'StudyDate'})

# Get lists of the id columns and the dependent variable columns
id_cols=['Study','Subject','StudyDay','StudyDate','File Name']

In [5]:
##need to adjust lists with the .1 and .2 notation
shape_list=['VoxelVolume','Maximum3DDiameter','Compactness2','MeshVolume'
,'MajorAxisLength','Sphericity','LeastAxisLength','Elongation'
,'Compactness1','SurfaceVolumeRatio','Maximum2DDiameterSlice'
,'Flatness','SurfaceArea','MinorAxisLength'
,'Maximum2DDiameterColumn','SphericalDisproportion','Maximum2DDiameterRow']

glcm_list=['JointAverage','Autocorrelation','JointEntropy','ClusterShade',
'MaximumProbability','Idmn','JointEnergy','Contrast',
'DifferenceEntropy','InverseVariance','DifferenceVariance',
'Idn','Idm','Correlation','SumAverage',
'SumEntropy','MCC','SumSquares','ClusterProminence',
'Imc2','Imc1','DifferenceAverage','Id','ClusterTendency']

gldm_list=['GrayLevelVariance','HighGrayLevelEmphasis'
,'DependenceEntropy','DependenceNonUniformity'
,'GrayLevelNonUniformity','SmallDependenceEmphasis'
,'SmallDependenceHighGrayLevelEmphasis','DependenceNonUniformityNormalized'
,'LargeDependenceEmphasis','LargeDependenceLowGrayLevelEmphasis'
,'DependenceVariance','LargeDependenceHighGrayLevelEmphasis'
,'SmallDependenceLowGrayLevelEmphasis','LowGrayLevelEmphasis']

firstorder_list=['InterquartileRange','Skewness'
,'Uniformity','Median','Energy'
,'RobustMeanAbsoluteDeviation','MeanAbsoluteDeviation'
,'StandardDeviation','TotalEnergy'
,'RootMeanSquared','90Percentile'
,'Minimum','Entropy','Range'
,'Variance','10Percentile','Kurtosis'
,'Maximum','Mean']

glrlm_list=['ShortRunLowGrayLevelEmphasis','GrayLevelVariance.1'
,'LowGrayLevelRunEmphasis','GrayLevelNonUniformityNormalized'
,'RunVariance','GrayLevelNonUniformity.1'
,'LongRunEmphasis','ShortRunHighGrayLevelEmphasis'
,'RunLengthNonUniformity','ShortRunEmphasis'
,'LongRunHighGrayLevelEmphasis','RunPercentage'
,'LongRunLowGrayLevelEmphasis','RunEntropy'
,'HighGrayLevelRunEmphasis','RunLengthNonUniformityNormalized']

glszm_list=['GrayLevelVariance.2','ZoneVariance'
,'GrayLevelNonUniformityNormalized.1','SizeZoneNonUniformityNormalized'
,'SizeZoneNonUniformity','GrayLevelNonUniformity.2'
,'LargeAreaEmphasis','SmallAreaHighGrayLevelEmphasis'
,'ZonePercentage','LargeAreaLowGrayLevelEmphasis'
,'LargeAreaHighGrayLevelEmphasis','HighGrayLevelZoneEmphasis'
,'SmallAreaEmphasis','LowGrayLevelZoneEmphasis'
,'ZoneEntropy','SmallAreaLowGrayLevelEmphasis']

ngtdm_list=['Coarseness','Complexity','Strength'
,'Contrast.1','Busyness']

In [6]:
# Confirm there aren't any repeats
rep_bool = len(set(shape_list+glcm_list+gldm_list+firstorder_list+glrlm_list+glszm_list+ngtdm_list)) != len(shape_list+glcm_list+gldm_list+firstorder_list+glrlm_list+glszm_list+ngtdm_list)
ninetynine(rep_bool,'repeat features in grouping lists')

FALSE: I've got 99 problems, but repeat features in grouping lists is not one


In [7]:
# Rename radiomic features to include class
for name in data.columns:
    if name in shape_list:
        data.rename(columns={name:name.split('.')[0]+'-shape'},inplace=True)
    elif name in glcm_list:
        data.rename(columns={name:name.split('.')[0]+'-glcm'},inplace=True)
    elif name in gldm_list:
        data.rename(columns={name:name.split('.')[0]+'-gldm'},inplace=True)
    elif name in firstorder_list:
        data.rename(columns={name:name.split('.')[0]+'-firstorder'},inplace=True)
    elif name in glrlm_list:
        data.rename(columns={name:name.split('.')[0]+'-glrlm'},inplace=True)
    elif name in glszm_list:
        data.rename(columns={name:name.split('.')[0]+'-glszm'},inplace=True)
    elif name in ngtdm_list:
        data.rename(columns={name:name.split('.')[0]+'-ngtdm'},inplace=True)
    else:
        print('Column not in list:',name)

Column not in list: Study
Column not in list: Subject
Column not in list: StudyDay
Column not in list: StudyDate
Column not in list: File Name
Column not in list: Class


In [8]:
# Feature name list for storage
var_cols=data.drop(id_cols+['Class'],axis=1).columns.tolist()

In [9]:
# Print column names
i=0
for name in data.columns:
    i += 1
    print (str(i)+') '+str(name))

1) Study
2) Subject
3) StudyDay
4) StudyDate
5) File Name
6) Class
7) VoxelVolume-shape
8) Maximum3DDiameter-shape
9) Compactness2-shape
10) MeshVolume-shape
11) MajorAxisLength-shape
12) Sphericity-shape
13) LeastAxisLength-shape
14) Elongation-shape
15) Compactness1-shape
16) SurfaceVolumeRatio-shape
17) Maximum2DDiameterSlice-shape
18) Flatness-shape
19) SurfaceArea-shape
20) MinorAxisLength-shape
21) Maximum2DDiameterColumn-shape
22) SphericalDisproportion-shape
23) Maximum2DDiameterRow-shape
24) JointAverage-glcm
25) Autocorrelation-glcm
26) JointEntropy-glcm
27) ClusterShade-glcm
28) MaximumProbability-glcm
29) Idmn-glcm
30) JointEnergy-glcm
31) Contrast-glcm
32) DifferenceEntropy-glcm
33) InverseVariance-glcm
34) DifferenceVariance-glcm
35) Idn-glcm
36) Idm-glcm
37) Correlation-glcm
38) SumAverage-glcm
39) SumEntropy-glcm
40) MCC-glcm
41) SumSquares-glcm
42) ClusterProminence-glcm
43) Imc2-glcm
44) Imc1-glcm
45) DifferenceAverage-glcm
46) Id-glcm
47) ClusterTendency-glcm
48) Gra

### Drop entries with errors

In [10]:
# # Remove a specific set of entries that have erroneous StudyDay values
data.loc[(data.Subject == 'B03942') & (data.StudyDate < '2020/06/01')] = None
data.dropna(how='all',inplace=True)
# Fix specific errors in group coding
data.loc[data.Subject == 'B03942','Class'] = 'Mock'
data.loc[data.Subject == 'G57L','Class'] = 'Mock' 
data.loc[data.Subject == 'G57N','Class'] = 'Infected' 

In [11]:
# Entries where subject ID does not match filename (confirmed we should keep these)
mask = (data.Subject != data['File Name'].str.split('_', n = 1, expand=True)[0])
data.loc[mask,['Subject','File Name','StudyDay','Class']]

,Subject,File Name,StudyDay,Class
84,B03942,B03041_20200609,-7,Mock
85,B03942,B03971_20200618,2,Mock
86,B03942,B03941_20200620,4,Mock
87,B03942,B03961_20200622,6,Mock
88,B03942,B03911_20200624,8,Mock


Remove entries at baseline or after day 8 time points

In [12]:
# Get a list of subjects that had a StudyDays 2, 4, and 6
subj_list = data.loc[data['StudyDay'].isin(['2','4','6']),'Subject']

# Drop all other subjects and extra rows of other time points (for simplicity)
ignore_days = ['BL','BL4','10','12','19','30']
data.drop(data[(data['StudyDay'].isin(ignore_days)) | (~data.Subject.isin(subj_list))].index, inplace = True)

In [13]:
# Check for rows with the same subj_id and StudyDay
ninetynine(len(data.loc[data.duplicated(subset=['Subject','StudyDay'])])>0,'rows with the same subj_id and StudyDay')

FALSE: I've got 99 problems, but rows with the same subj_id and StudyDay is not one


In [14]:
# Separate time points into separate columns
data = data.set_index(['Subject','StudyDay']).unstack()

Calculate the change from pre-exposure

In [15]:
for var in var_cols:
    # Collapse pre-exposure values (should have one per subject)
    data.loc[:,(var,'pre')] = data.loc[:,(var,['-5','-6','-7'])].mean(axis = 1, skipna = True)

    # Calculate the change from pre-exposure
    data.loc[:,(var,'pre_delta')] = data.loc[:,(var,'pre')]-data.loc[:,(var,'pre')] #should be all 0
    data.loc[:,(var,'2_delta')] = data.loc[:,(var,'2')]-data.loc[:,(var,'pre')]
    data.loc[:,(var,'4_delta')] = data.loc[:,(var,'4')]-data.loc[:,(var,'pre')]
    data.loc[:,(var,'6_delta')] = data.loc[:,(var,'6')]-data.loc[:,(var,'pre')]
    data.loc[:,(var,'8_delta')] = data.loc[:,(var,'8')]-data.loc[:,(var,'pre')]


In [16]:
data.head()

Study                                             StudyDate  \
StudyDay     -5     -6     -7      2      4      6      8          -5   
Subject                                                                 
B03757      NaN  64-01    NaN  64-01  64-01  64-01  64-01         NaN   
B03781      NaN    NaN  64-02  64-02  64-02  64-02  64-02         NaN   
B03819      NaN    NaN  64-02  64-02  64-02  64-02  64-02         NaN   
B03828      NaN    NaN  64-02  64-02  64-02  64-02  64-02         NaN   
B03843    64-01    NaN    NaN  64-01  64-01  64-01  64-01  2020/03/16   

                                  ... Contrast-ngtdm                      \
StudyDay          -6          -7  ...        2_delta   4_delta   6_delta   
Subject                           ...                                      
B03757    2020/03/16         NaN  ...      -0.012990  0.003805  0.000367   
B03781           NaN  2020/06/10  ...      -0.013450 -0.012341 -0.014196   
B03819           NaN  2020/06/10  ...      -0.004805 -0.021873 -0.011045   
B03828           NaN  2020/06/09  ...       0.006496 -0.000789  0.002532   
B03843           NaN         NaN  ...      -0.004060 -0.004307 -0.003359   

                   Busyness-ngtdm                                    \
StudyDay   8_delta            pre pre_delta     2_delta     4_delta   
Subject                                                               
B03757   -0.004608    1200.810742       0.0 -965.677485 -857.330332   
B03781   -0.000580     759.886160       0.0 -334.435885 -254.084988   
B03819   -0.013539     898.182254       0.0 -454.862459 -761.628902   
B03828    0.002675     803.870337       0.0  185.062826    3.483671   
B03843   -0.001183    1727.274784       0.0 -241.882499 -115.547354   

                                  
StudyDay     6_delta     8_delta  
Subject                           
B03757   -857.721661 -771.388894  
B03781   -110.322327   21.745617  
B03819   -561.964909 -417.221327  
B03828    279.038216  313.243609  
B03843     53.090560 -222.430157  

[5 rows x 1471 columns]

In [17]:
# Copy the class to the new columns from the original columns

# stack and unstack so that missing columns fill in with null
data = data.stack().unstack()

time_points=['pre','2','4','6','8','pre_delta','2_delta','4_delta','6_delta','8_delta']
delta_time=['pre_delta','2_delta','4_delta','6_delta','8_delta']
# back-fill and foward-fill incase the column order changes
data.loc[:,('Class',time_points)] = data.loc[:,('Class',time_points)].fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)

# Reformat for readability
data = data.stack()

### View the data

In [18]:
# Uncomment for different popular views of the data
#data#.describe()
#data.unstack()['Strength']

#data.groupby(['StudyDay','Class']).describe()

#data.loc[data['StudyDay'] == -6]
#data.reset_index().loc[data['Subject'] == 'B03942',['Subject','StudyDay','Class']]
#data.loc[(data.Class == 'Mock') & (data.StudyDay == '2')]
#data.loc[data['Class'] == 'Infected',['Subject','Class']]
#data.reset_index()
#data.describe()
#data.loc[(data.Class == 'Mock') & (data.StudyDay == '2'),'Subject']

### Save resulting tables

In [19]:
# Save full table (no error values)
data.to_csv(os.path.join(PATH,'tables','data_radio.csv'))

# Save simplified table (only delta timepoints)
data = data.reset_index()
data_deltaOnly = data.drop(data[data['StudyDay'].isin(['-5','-6','-7','2','4','6','8','pre','pre_delta'])].index)
data_deltaOnly.set_index(['Subject','StudyDay']).to_csv(os.path.join(PATH,'tables','data_radio_delta.csv'))

### Exclude variables based on domain-specific feature screening criteria

In [20]:
var_MCExclude=['SurfaceVolumeRatio-shape','Maximum2DDiameterSlice-shape'
,'MaximumProbability-glcm','DifferenceEntropy-glcm'
,'InverseVariance-glcm','SmallDependenceEmphasis-gldm'
,'DependenceNonUniformityNormalized-gldm','LargeDependenceEmphasis-gldm'
,'LargeDependenceLowGrayLevelEmphasis-gldm','DependenceVariance-gldm'
,'SmallDependenceLowGrayLevelEmphasis-gldm'
,'Minimum-firstorder'
,'RunVariance-glrlm','LongRunEmphasis-glrlm'
,'ShortRunEmphasis-glrlm','RunPercentage-glrlm'
,'LongRunLowGrayLevelEmphasis-glrlm','RunLengthNonUniformityNormalized-glrlm'
,'ZonePercentage-glszm','LowGrayLevelZoneEmphasis-glszm'
,'SmallAreaLowGrayLevelEmphasis-glszm']

In [21]:
# Check that listed variables are all in the dataset
for var in var_MCExclude:
    if not var in data.columns.tolist():
        print('Not in dataset:',var)

In [22]:
# Get a list of subset of variables after domain-specific feature screening
var_cols = [ var for var in var_cols if not var in var_MCExclude]

# Write to status message file, change this anytime this part of the analysis changes
status='radioExcl_MC;'
with open(os.path.join('..','config','analysis_status','u_radio.txt'),'w') as out_file:
    out_file.write(status)

#### Save variables

In [23]:
# Saving the objects:
with open('../config/lists_radio.pkl', 'wb') as f:
    pickle.dump([id_cols,var_cols,time_points,delta_time], f)

In [24]:
# Print column names
i=0
for name in var_cols:
    i += 1
    print (str(i)+') '+str(name))

1) VoxelVolume-shape
2) Maximum3DDiameter-shape
3) Compactness2-shape
4) MeshVolume-shape
5) MajorAxisLength-shape
6) Sphericity-shape
7) LeastAxisLength-shape
8) Elongation-shape
9) Compactness1-shape
10) Flatness-shape
11) SurfaceArea-shape
12) MinorAxisLength-shape
13) Maximum2DDiameterColumn-shape
14) SphericalDisproportion-shape
15) Maximum2DDiameterRow-shape
16) JointAverage-glcm
17) Autocorrelation-glcm
18) JointEntropy-glcm
19) ClusterShade-glcm
20) Idmn-glcm
21) JointEnergy-glcm
22) Contrast-glcm
23) DifferenceVariance-glcm
24) Idn-glcm
25) Idm-glcm
26) Correlation-glcm
27) SumAverage-glcm
28) SumEntropy-glcm
29) MCC-glcm
30) SumSquares-glcm
31) ClusterProminence-glcm
32) Imc2-glcm
33) Imc1-glcm
34) DifferenceAverage-glcm
35) Id-glcm
36) ClusterTendency-glcm
37) GrayLevelVariance-gldm
38) HighGrayLevelEmphasis-gldm
39) DependenceEntropy-gldm
40) DependenceNonUniformity-gldm
41) GrayLevelNonUniformity-gldm
42) SmallDependenceHighGrayLevelEmphasis-gldm
43) LargeDependenceHighGra